In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import argparse

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import itertools
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

transform_train = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


transform_test = transforms.Compose([transforms.Resize(320),
        transforms.RandomCrop(299), transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


train_path = '../datasets/nsfw/train_balanced/'
trainset = torchvision.datasets.ImageFolder(train_path, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=16, shuffle=True, num_workers=2)
test_path = '../datasets/nsfw/val_balanced/'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# Get pretrained wide resnet model
net = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
net.fc = nn.Linear(512, 2, bias=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [9]:
net = net.to(device)
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs, targets.to(device)
        if(np.random.uniform(0, 1)<0.3):
            inputs = inputs + torch.rand(inputs.shape)
            inputs = torch.clamp(inputs, max=1, min=0)
        inputs = inputs.to(device) 
        outputs = net(inputs)

        loss = criterion(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        lr = optimizer.param_groups[0]["lr"]

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

#         progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d) | lr: %.5f'
#                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, lr))

        
        

def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
    
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

#             progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            acc = 100.*correct/total
    return acc

In [11]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001, 
                              weight_decay=2e-4,
                              momentum=0.9) 
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40000, 80000], gamma=0.1)#4000
criterion = nn.CrossEntropyLoss().to(device)

In [12]:
best_test_acc = 0  
i = 0
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    acc = test(epoch)
    if acc>=best_test_acc:
        #print('==> Saving model to: ', args.save_path)
        torch.save(net.state_dict(), 'NSFW_model/ResNet_NSFW_'+str(i)+'.pth')
        print(acc)
        best_test_acc = acc
    i += 1



Epoch: 0


KeyboardInterrupt: 

In [17]:
import os
cwd = os.getcwd()
print(cwd)

/workspace/adv_robustness/cyberbullying_purification/samuel_src/non_robust_classifiers


In [31]:
net = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
net.to(device)
checkpoint = torch.load('../auxiliary_models/ResNet_NSFW.pth')
model.load_state_dict(checkpoint, strict=False)
# transform_test = transforms.Compose([transforms.Resize(320),
#         transforms.RandomCrop(299), transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_test = transforms.Compose([transforms.ToTensor()])
good_img_transform = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

classes = {0:'neutral', 1:'porn'}

test_path = '../datasets/nsfw_attacked_simple/test'
testset = torchvision.datasets.ImageFolder(test_path, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=True, num_workers=2)

model.to(device)
model.eval()
model.load_state_dict(checkpoint, strict=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [33]:
correct = 0
total_samples = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs, targets
        
        #print(targets)
        
        label = targets.item()
        outputs = model(inputs.to(device))
        _, predicted = outputs.max(1)
        pred_val = predicted.item()
    
        total_samples += 1
        if pred_val == label:
            correct += 1
        
        if total_samples > 50:
            break
print(correct / total_samples)

0.5490196078431373


In [38]:
images, labels = next(itertools.islice(testloader, 1, None))
print(images.min())

tensor(0.)
